In [ ]:
# Импорт библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
df = pd.read_csv("Datasets\\sepsis-diagnoses.xls")

In [ ]:
df.info()

In [ ]:
df.head(10)

In [ ]:
# Выбор нужных признаков
selected_features = [
    "HR",
    "O2Sat",
    "Temp",
    "MAP",
    "Resp",
    "Glucose",
    "Lactate",
    "WBC",
    "Hgb",
    "Platelets",
    "Age",
    "Gender",
    "SepsisLabel",
]

In [ ]:
# Отбор нужных признаков из данных
data_selected = df[selected_features]

In [ ]:
# Преобразование данных в массив NumPy
data_array = data_selected.values

In [ ]:
# Масштабирование данных
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_array)

In [ ]:
# Разделение данных на признаки (X) и целевую переменную (y)
X = scaled_data[:, :-1]  # Признаки
y = scaled_data[:, -1]  # Целевая переменная

In [ ]:
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)  # Пример разделения на 80% обучающих данных и 20% тестовых данных

In [ ]:
# Преобразование данных для модели LSTM
def prepare_data_for_lstm(data, history_size):
    features = []
    labels = []
    for i in range(len(data) - history_size):
        features.append(data[i : i + history_size, :])
        labels.append(data[i + history_size, -1])
    return np.array(features), np.array(labels)

In [ ]:
# Параметры модели и обучения
history_size = 50  # Длина истории (количество временных точек в прошлом)
batch_size = 64
epochs = 10

In [ ]:
# Подготовка данных для обучения модели LSTM
x_train_lstm, y_train_lstm = prepare_data_for_lstm(X_train, history_size)
x_test_lstm, y_test_lstm = prepare_data_for_lstm(X_test, history_size)

In [ ]:
# Создание модели LSTM
model_lstm = Sequential(
    [
        LSTM(
            64,
            input_shape=(x_train_lstm.shape[1], x_train_lstm.shape[2]),
            return_sequences=True,
        ),
        Dropout(0.2),
        LSTM(64),
        Dense(1),
    ]
)

In [ ]:
# Компиляция модели LSTM
model_lstm.compile(optimizer="adam", loss="mse")

In [ ]:
# Обучение модели LSTM
history_lstm = model_lstm.fit(
    x_train_lstm,
    y_train_lstm,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(x_test_lstm, y_test_lstm),
)

In [ ]:
# Оценка модели LSTM
test_loss_lstm = model_lstm.evaluate(x_test_lstm, y_test_lstm)
print("Test Loss LSTM:", test_loss_lstm)

In [ ]:
from sklearn.metrics import r2_score

# Оценка модели LSTM
test_loss_lstm = model_lstm.evaluate(x_test_lstm, y_test_lstm)
print("Test Loss LSTM:", test_loss_lstm)

# Прогнозирование на тестовых данных
y_pred_lstm = model_lstm.predict(x_test_lstm)

# Вычисление R2 score
r2_score_lstm = r2_score(y_test_lstm, y_pred_lstm)
print("R2 Score LSTM:", r2_score_lstm)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Оценка модели LSTM
test_loss_lstm = model_lstm.evaluate(x_test_lstm, y_test_lstm)
print("Test Loss LSTM:", test_loss_lstm)

# Прогнозирование на тестовых данных
y_pred_lstm = model_lstm.predict(x_test_lstm)

# Вычисление MAE
mae_lstm = mean_absolute_error(y_test_lstm, y_pred_lstm)
print("MAE LSTM:", mae_lstm)

# Вычисление MSE
mse_lstm = mean_squared_error(y_test_lstm, y_pred_lstm)
print("MSE LSTM:", mse_lstm)

# Вычисление R2 score
r2_score_lstm = r2_score(y_test_lstm, y_pred_lstm)
print("R2 Score LSTM:", r2_score_lstm)

In [ ]:
# ROC AUC
roc_auc_lstm = roc_auc_score(y_test_lstm, y_pred_lstm)

# Построение ROC кривой
fpr_lstm, tpr_lstm, thresholds_lstm = roc_curve(y_test_lstm, y_pred_lstm)
plt.figure(figsize=(8, 6))
plt.plot(fpr_lstm, tpr_lstm, label=f"ROC AUC = {roc_auc_lstm:.2f}")
plt.plot([0, 1], [0, 1], linestyle="--", color="red")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve - LSTM")
plt.legend()
plt.show()

# График совпадения модели при обучении
plt.figure(figsize=(8, 6))
plt.plot(y_test_lstm, label="True Values")
plt.plot(y_pred_lstm, label="Predicted Values")
plt.xlabel("Samples")
plt.ylabel("Values")
plt.title("Model Prediction vs True Values - LSTM")
plt.legend()
plt.show()

In [ ]:
# Выбор клинических показателей для создания композитного индекса
selected_columns = ["HR", "O2Sat", "Temp", "MAP", "Resp"]

In [ ]:
# Создание композитного индекса
composite_index = df[selected_columns].mean(axis=1)

In [ ]:
# Добавление композитного индекса в исходные данные
df["CompositeIndex"] = composite_index

In [ ]:
# Проверка результатов
print(df.head())

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Определение признаков и целевой переменной
features = ["HR", "MAP", "O2Sat", "Temp"]
target = "CompositeIndex"

In [ ]:
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Обучение модели линейной регрессии
model = LinearRegression()
model.fit(X_train_scaled, y_train)

In [ ]:
# Предсказание на тестовом наборе
y_pred = model.predict(X_test_scaled)

In [ ]:
# Оценка качества модели
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

In [ ]:
# Вычисление средней абсолютной ошибки (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

# Вычисление коэффициента детерминации (R^2)
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)